### Textual explanation creation


In this notebook we will:

1️⃣ Load a model


2️⃣ Generate paths from explainable methods like PGPR


3️⃣ Generate textual explanations through templates

**Load libraries**

In [9]:
from hopwise.quick_start import load_data_and_model
from hopwise.utils.case_study import full_sort_scores

**Define model checkpoint 📍**

In [10]:
checkpoint = "saved/PGPR-Mar-17-2025_14-58-05.pth"

**Define templates 📍**

Consider a path in the form of: `'user', 'relation1', 'entity1', 'relation2', 'entity2', 'relation3', 'product'`

In [ ]:
templates = [
    "{product} is recommend to you because you {relation1} {entity1} also {relation2} by {entity2}",
]

**Load model checkpoint with its configuration**

In [12]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file=checkpoint)

17 Mar 23:42    INFO  
General Hyper Parameters:
gpu_id = 1
use_gpu = True
seed = 2024
state = INFO
reproducibility = True
data_path = /home/recsysdatasets/ml-1m_small
checkpoint_dir = saved
show_progress = True
save_dataset = True
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = NDCG@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hy

**Load mappings**

In [13]:
# we map back the entities and users to their original values in the dataset
def pid2item(x): return dataset.id2token(dataset.iid_field, x)
def eid2entity(x): return dataset.id2token(dataset.entity_field, x)
def uid2user(x): return dataset.id2token(dataset.uid_field, x)
def rid2relation(x): return dataset.id2token(dataset.relation_field, x)

**Specify for which users provide explanations 👤**

In [14]:
#provide explanations for ORIGINAL user 196 and user 186. 
#Consider that the remapped uids are passed to the model, not the ones you provided!!
uid_series = dataset.token2id(dataset.uid_field, ["196", "186"])

**Get explanations from model inference output 🤩**

In [15]:
# explanations is a pandas dataframe with columns: uid, product, score, explanation (the produced path)
explanations = full_sort_scores(
    uid_series, model, test_data, device=config["device"], explain=True)

**Fill templates 🤔**

In [16]:
#  fill templates
e_type2mapping = {'U': uid2user, 'E': eid2entity, 'I': pid2item, 'R': rid2relation}

for i, template in enumerate(templates):
    filled_template = list()
    for path in explanations['path']:
        user, relation1, entity1, relation2, entity2, relation3, product = path

        # remap to original entities and items. relations are already mapped back
        user = e_type2mapping[user[0]](int(user[1:]))
        relation = e_type2mapping[relation1[0]](int(relation1[1:]))
        entity1 = e_type2mapping[entity1[0]](int(entity1[1:]))
        entity2 = e_type2mapping[entity2[0]](int(entity2[1:]))
        
        filled_template.append(template.format(
            user=user,
            relation1=relation1,
            entity1=entity1,
            relation2=relation2,
            entity2=entity2,
            relation3=relation3,
            product=product
            )
        )
    # we create a new column for each template
    explanations[f'template{i}'] = filled_template

**Show textual explanations 🚀**

In [17]:
explanations

,user,product,score,path,template0
0,196,286,6.921911,"[U196, R12, I2231, R12, U3681, R12, I286]",I286 is recommend to you because you R12 2458 ...
1,196,120,6.975404,"[U196, R12, I949, R11, E10360, R11, I120]",I120 is recommend to you because you R12 2993 ...
2,196,485,7.031409,"[U196, R12, I2279, R12, U4480, R12, I485]",I485 is recommend to you because you R12 2862 ...
3,196,121,7.224464,"[U196, R12, I461, R12, U5366, R12, I121]",I121 is recommend to you because you R12 3265 ...
4,196,1073,7.470816,"[U196, R12, I461, R12, U4862, R12, I1073]",I1073 is recommend to you because you R12 3265...
5,196,674,7.557087,"[U196, R12, I949, R12, U5793, R12, I674]",I674 is recommend to you because you R12 2993 ...
6,196,1901,7.591155,"[U196, R12, I989, R12, U5530, R12, I1901]",I1901 is recommend to you because you R12 26 a...
7,196,921,8.076472,"[U196, R12, I202, R12, U3588, R12, I921]",I921 is recommend to you because you R12 3527 ...
8,196,1895,8.364883,"[U196, R12, I586, R12, U5907, R12, I1895]",I1895 is recommend to you because you R12 904 ...
9,196,300,9.885301,"[U196, R12, I411, R12, U5180, R12, I300]",I300 is recommend to you because you R12 1682 ...
